In [1]:
from typing import List, Dict, Tuple

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/clay/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Model and Tokenizer

In [3]:
pretrained_model_name_or_path = "./models/google--gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path, torch_dtype=torch.bfloat16)
model.to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  9.29it/s]


Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), 

## Set Banned Words

In [4]:
banned_words = ["talk", "listen", "fuck you"]

In [5]:
import re

new_token_candidates = []
new_token_id_candidates = []

# Define a pattern to filter out unwanted excessive whitespace tokens (e.g., multiple tabs or newlines)
invalid_token_pattern = re.compile(r'^[\t\n\r\f\v\s]*$')  # Matches tokens that are made of 3 or more whitespace characters

# Initial matching of tokens that start with banned word prefixes
for token, token_id in tokenizer.vocab.items():
    token_stripped = token.strip()
    if len(token_stripped) == 0 or (len(token_stripped) == 1 and token_stripped == "▁"): 
        continue

    if token_stripped[0] == "▁":
        token_stripped = token_stripped[1:]

    for banned_word in banned_words:
        if banned_word.startswith(token_stripped):
            new_token_candidates.append([token])  # Store as a list for consistency
            new_token_id_candidates.append([token_id])

print("Filtered First Candidates:", new_token_candidates)
print("Filtered First IDs:", new_token_id_candidates)

# Initialize final routes for fully matched banned words
final_routes = []
final_id_routes = []
for new_token, new_token_id in zip(new_token_candidates, new_token_id_candidates):
    new_token_str = "".join(new_token).strip()

    if new_token_str in banned_words or (new_token_str[0] == "▁" and new_token_str[1:] in banned_words):
        final_routes.append(new_token)
        final_id_routes.append(new_token_id)

print("(beginning) Final Routes:", final_routes)
print("(beginning) Final ID Routes:", final_id_routes)

# Iteratively expand candidates using BFS-like approach
while new_token_candidates:
    curr_token_candidates = new_token_candidates
    curr_token_id_candidates = new_token_id_candidates
    new_token_candidates = []
    new_token_id_candidates = []

    # Iterate over vocabulary and expand each candidate
    for token, token_id in tokenizer.vocab.items():
        token_stripped = token.strip()
        if len(token_stripped) == 0 or (len(token_stripped) == 1 and token_stripped == "▁") or re.findall(r"\▁{2,100}", token_stripped):
            continue

        # Skip tokens that consist of excessive whitespace or control characters
        if invalid_token_pattern.match(token_stripped):
            continue

        for candidate_token, candidate_token_id in zip(curr_token_candidates, curr_token_id_candidates):
            curr_token = candidate_token + [token]
            curr_token_ids = candidate_token_id + [token_id]

            curr_token_str = "".join([token.replace("▁", " ") for token in curr_token]).strip()
            # if curr_token_str[0] == "▁":
            #     curr_token_str = curr_token_str[1:]

            # Check if the current token combination matches or is a prefix of any banned word
            for banned_word in banned_words:
                if curr_token_str == banned_word:
                    # Full match found, add to final routes
                    final_routes.append(curr_token)
                    final_id_routes.append(curr_token_ids)
                elif banned_word.startswith(curr_token_str):
                    # Partial match, keep expanding this candidate
                    new_token_candidates.append(curr_token)
                    new_token_id_candidates.append(curr_token_ids)

    print(final_routes)

print("Final Routes:", final_routes)
print("Final ID Routes:", final_id_routes)

Filtered First Candidates: [['▁t'], ['▁tal'], ['fuck'], ['▁f'], ['tal'], ['f'], ['ta'], ['▁fuck'], ['▁list'], ['li'], ['▁fu'], ['▁li'], ['▁ta'], ['l'], ['listen'], ['list'], ['▁liste'], ['▁l'], ['▁listen'], ['fu'], ['▁talk'], ['lis'], ['liste'], ['t'], ['▁fuc'], ['talk'], ['▁lis']]
Filtered First IDs: [[474], [3412], [34024], [517], [3559], [235266], [516], [7935], [1889], [515], [4936], [702], [3586], [235257], [18998], [1701], [32165], [533], [10724], [12819], [5063], [15063], [44003], [235251], [79433], [33085], [23966]]
(beginning) Final Routes: [['listen'], ['▁listen'], ['▁talk'], ['talk']]
(beginning) Final ID Routes: [[18998], [10724], [5063], [33085]]
[['listen'], ['▁listen'], ['▁talk'], ['talk'], ['▁tal', 'k'], ['tal', 'k'], ['li', 'sten'], ['▁li', 'sten'], ['ta', 'lk'], ['▁ta', 'lk'], ['lis', 'ten'], ['▁lis', 'ten'], ['▁t', 'alk'], ['t', 'alk'], ['▁liste', 'n'], ['liste', 'n'], ['▁list', 'en'], ['list', 'en'], ['l', 'isten'], ['▁l', 'isten'], ['fuck', '▁you'], ['▁fuck', '▁you

In [16]:
for items, ids in zip(final_routes, final_id_routes):
    print(items, ids)

['listen'] [18998]
['▁listen'] [10724]
['▁talk'] [5063]
['talk'] [33085]
['▁tal', 'k'] [3412, 235273]
['tal', 'k'] [3559, 235273]
['li', 'sten'] [515, 5547]
['▁li', 'sten'] [702, 5547]
['ta', 'lk'] [516, 26159]
['▁ta', 'lk'] [3586, 26159]
['lis', 'ten'] [15063, 965]
['▁lis', 'ten'] [23966, 965]
['▁t', 'alk'] [474, 2071]
['t', 'alk'] [235251, 2071]
['▁liste', 'n'] [32165, 235254]
['liste', 'n'] [44003, 235254]
['▁list', 'en'] [1889, 479]
['list', 'en'] [1701, 479]
['l', 'isten'] [235257, 17071]
['▁l', 'isten'] [533, 17071]
['fuck', '▁you'] [34024, 692]
['▁fuck', '▁you'] [7935, 692]
['ta', 'l', 'k'] [516, 235257, 235273]
['▁ta', 'l', 'k'] [3586, 235257, 235273]
['▁t', 'al', 'k'] [474, 492, 235273]
['t', 'al', 'k'] [235251, 492, 235273]
['l', 'i', 'sten'] [235257, 235252, 5547]
['▁l', 'i', 'sten'] [533, 235252, 5547]
['▁t', 'a', 'lk'] [474, 235250, 26159]
['t', 'a', 'lk'] [235251, 235250, 26159]
['l', 'is', 'ten'] [235257, 502, 965]
['▁l', 'is', 'ten'] [533, 502, 965]
['li', 's', 'ten'] [

## Normal Decoding

In [17]:
input_text = "Can we talk?"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

In [18]:
def custom_generate(input_ids: torch.Tensor, max_length: int = 50) -> str:
    for _ in range(max_length):
        # Generate new tokens
        outputs = model(input_ids.to(device), return_dict=True, use_cache=True)
        logits = outputs.logits[:, -1, :]
        
        new_generated_token_id = torch.argmax(logits, dim=-1)

        if new_generated_token_id == tokenizer.eos_token_id:
            break

        input_ids = torch.cat((input_ids, new_generated_token_id.unsqueeze(0)), dim=-1)

    return tokenizer.decode(input_ids[0], skip_special_tokens=False)

In [19]:
print(custom_generate(input_ids=inputs.input_ids))

<bos>Can we talk?

I'm here to listen and help in any way I can. 

What's on your mind? 
<end_of_turn>


## FSM

In [20]:
class FSMProcessor:
    def __init__(self, special_token_ids_list: List[List[int]], end_state: int = -1) -> None:
        self.end_state = end_state
        self.next_state = 1
        self.curr_state = 0
        self.fsm = {}
        self.special_words = []

        # Track partial matches
        self.partial_match_state = None
        self.partial_tokens = []

        self.update_group(special_token_ids_list)

    def update(self, special_token_ids: List[int]) -> None:
        curr_state = 0

        for idx, special_token_id in enumerate(special_token_ids):
            if curr_state not in self.fsm:
                self.fsm[curr_state] = []

            state2id = [items[0] for items in self.fsm[curr_state]]
            if special_token_id not in state2id:
                if idx == len(special_token_ids) - 1:
                    self.fsm[curr_state].append([special_token_id, self.end_state])
                else:
                    self.fsm[curr_state].append([special_token_id, self.next_state])
                    curr_state = self.next_state
                    self.next_state += 1
            else:
                for fsm_idx in range(len(self.fsm[curr_state])):
                    if special_token_id == self.fsm[curr_state][fsm_idx][0] and idx == len(special_token_ids) - 1:
                        self.fsm[curr_state][fsm_idx][1] = self.end_state
                        break
                    elif special_token_id == self.fsm[curr_state][fsm_idx][0]:
                        curr_state = self.fsm[curr_state][fsm_idx][1]
                        break

    def update_group(self, special_token_ids_list: List[List[int]]) -> None:
        for special_token_ids in special_token_ids_list:
            self.update(special_token_ids=special_token_ids)

    def get_fsm_data(self) -> Dict[str, List[Tuple[int, int]]]:
        return self.fsm
    
    def detect(self, token: int) -> bool:
        """
        Detect if the current token leads to a sensitive sequence.
        Updates the current state and returns True if it reaches the end state.
        """
        if self.curr_state in self.fsm:
            for transition in self.fsm[self.curr_state]:
                if transition[0] == token:
                    self.curr_state = transition[1]

                    # If the current state reaches the end state
                    return self.curr_state == self.end_state
        
        # If the token does not match, reset the current state
        self.curr_state = 0
        return False

In [21]:
fsm_processor = FSMProcessor(special_token_ids_list=final_id_routes)
fsm_processor.get_fsm_data()

{0: [[18998, -1],
  [10724, -1],
  [5063, -1],
  [33085, -1],
  [3412, 1],
  [3559, 2],
  [515, 3],
  [702, 4],
  [516, 5],
  [3586, 6],
  [15063, 7],
  [23966, 8],
  [474, 9],
  [235251, 10],
  [32165, 11],
  [44003, 12],
  [1889, 13],
  [1701, 14],
  [235257, 15],
  [533, 16],
  [34024, 17],
  [7935, 18],
  [4936, 47],
  [12819, 49],
  [79433, 51],
  [517, 53],
  [235266, 55]],
 1: [[235273, -1]],
 2: [[235273, -1]],
 3: [[5547, -1], [235256, 29], [2855, 35], [490, 43]],
 4: [[5547, -1], [235256, 30], [2855, 36], [490, 44]],
 5: [[26159, -1], [235257, 19]],
 6: [[26159, -1], [235257, 20]],
 7: [[965, -1], [488, 37], [235251, 45]],
 8: [[965, -1], [488, 38], [235251, 46]],
 9: [[2071, -1], [492, 21], [235250, 25]],
 10: [[2071, -1], [492, 22], [235250, 26]],
 11: [[235254, -1]],
 12: [[235254, -1]],
 13: [[479, -1], [235249, 39]],
 14: [[479, -1], [235249, 40]],
 15: [[17071, -1], [235252, 23], [502, 27], [3671, 33], [694, 41]],
 16: [[17071, -1], [235252, 24], [502, 28], [3671, 34], 

In [22]:
def custom_generate_with_fsm_filter(
    input_ids: torch.Tensor,
    fsm_processor: FSMProcessor,
    max_length: int = 20,
) -> str:
    # Historical mask list used to record masked tokens at each decoding step
    masked_tokens_history = {}
    past_key_values = None
    steps = 0

    while steps < max_length:
        steps += 1

        # Generate new token with kv cache
        outputs = model(input_ids, past_key_values=past_key_values, return_dict=True, use_cache=True)
        logits = outputs.logits[:, -1, :]

        # Update kv cache
        past_key_values = outputs.past_key_values

        # Check if there are already masked tokens at the current step
        if steps in masked_tokens_history:
            for masked_token_id in masked_tokens_history[steps]:
                logits[:, masked_token_id] = -float("inf")
        else:
            masked_tokens_history[steps] = set()

        # Decode the generated token
        generated_token_id = torch.argmax(logits, dim=-1).item()
        combined_ids = torch.cat((input_ids, torch.tensor([[generated_token_id]], device=input_ids.device)), dim=-1)

        # Check FSM for sensitive sequences
        if fsm_processor.detect(generated_token_id):
            # Detected a sensitive sequence, initiate rollback
            rollback_length = fsm_processor.partial_match_state + 1 if fsm_processor.partial_match_state is not None else 1
            steps = steps - rollback_length + 1
            rollbacks_ids = combined_ids[:, :-rollback_length]
            input_ids = rollbacks_ids
            print(f"Rollback detected. Rolling back from step {steps + rollback_length} to step {steps}")

            # Reset FSM state
            fsm_processor.curr_state = 0
            fsm_processor.partial_match_state = None

            # Reset past_key_values when rolling back
            past_key_values = None

            # Recalculate logits based on rolled-back sequence
            outputs = model(input_ids, return_dict=True, use_cache=True)
            logits = outputs.logits[:, -1, :]
            past_key_values = outputs.past_key_values

            # Mask the first token of the sensitive sequence
            first_token_id = generated_token_id
            print(f"Masking token id: {first_token_id}, Masking token: {tokenizer.decode(first_token_id)}")

            # Update the historical mask list to record the token at this step
            masked_tokens_history[steps].add(first_token_id)

            for masked_token_id in masked_tokens_history[steps]:
                logits[:, masked_token_id] = -float("inf")

            # Generate the token again after masking
            generated_token_id = torch.argmax(logits, dim=-1).item()

        # Update input_ids with the generated token
        input_ids = torch.cat((input_ids, torch.tensor([[generated_token_id]], device=input_ids.device)), dim=1)

        print(f"Step {steps}: ID: {generated_token_id} Generated token: {tokenizer.decode(generated_token_id)}")

        if generated_token_id == tokenizer.eos_token_id:
            break

    return tokenizer.decode(input_ids[0], skip_special_tokens=False)

In [23]:
print(custom_generate_with_fsm_filter(
    input_ids=inputs.input_ids,
    fsm_processor=fsm_processor,
    max_length=50,
))

Step 1: ID: 109 Generated token: 


Step 2: ID: 235285 Generated token: I
Step 3: ID: 235303 Generated token: '
Step 4: ID: 235262 Generated token: m
Step 5: ID: 1517 Generated token:  here
Step 6: ID: 577 Generated token:  to
Rollback detected. Rolling back from step 8 to step 7
Masking token id: 10724, Masking token:  listen
Step 7: ID: 1707 Generated token:  help
Step 8: ID: 692 Generated token:  you
Step 9: ID: 675 Generated token:  with
Step 10: ID: 9550 Generated token:  whatever
Step 11: ID: 692 Generated token:  you
Step 12: ID: 1476 Generated token:  need
Step 13: ID: 235265 Generated token: .
Step 14: ID: 235248 Generated token:  
Step 15: ID: 109 Generated token: 


Step 16: ID: 5958 Generated token: Please
Step 17: ID: 3337 Generated token:  tell
Step 18: ID: 682 Generated token:  me
Step 19: ID: 1212 Generated token:  what
Step 20: ID: 235303 Generated token: '
Step 21: ID: 235256 Generated token: s
Step 22: ID: 611 Generated token:  on
Step 23: ID: 861 Generated token:  y

In [24]:
response = custom_generate_with_fsm_filter(
    input_ids=inputs.input_ids,
    fsm_processor=fsm_processor,
    max_length=50,
)
print(response)

Step 1: ID: 109 Generated token: 


Step 2: ID: 235285 Generated token: I
Step 3: ID: 235303 Generated token: '
Step 4: ID: 235262 Generated token: m
Step 5: ID: 1517 Generated token:  here
Step 6: ID: 577 Generated token:  to
Rollback detected. Rolling back from step 8 to step 7
Masking token id: 10724, Masking token:  listen
Step 7: ID: 1707 Generated token:  help
Step 8: ID: 692 Generated token:  you
Step 9: ID: 675 Generated token:  with
Step 10: ID: 9550 Generated token:  whatever
Step 11: ID: 692 Generated token:  you
Step 12: ID: 1476 Generated token:  need
Step 13: ID: 235265 Generated token: .
Step 14: ID: 235248 Generated token:  
Step 15: ID: 109 Generated token: 


Step 16: ID: 5958 Generated token: Please
Step 17: ID: 3337 Generated token:  tell
Step 18: ID: 682 Generated token:  me
Step 19: ID: 1212 Generated token:  what
Step 20: ID: 235303 Generated token: '
Step 21: ID: 235256 Generated token: s
Step 22: ID: 611 Generated token:  on
Step 23: ID: 861 Generated token:  y

In [15]:
print(response)

<bos>Can we talk?

I'm here to help you with whatever you need. 

Please tell me what's on your mind. 😊
<end_of_turn><eos>
